In [1]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from IPython.display import clear_output

# Set working path
path = '/Users/martinbogaert/Desktop/NBA Data Analysis/nba_ai_awards/'

print('MAKE SURE YOU CHANGE THE WEEK VARIABLE')

MAKE SURE YOU CHANGE THE WEEK VARIABLE


In [2]:
week = 20 # CHANGE THIS
date = datetime.now().strftime('%d %B %Y')
time = datetime.now().strftime('%H:%M:%S')
print(date + ' ; ' + time)

07 March 2023 ; 16:09:23


In [3]:
features = ['PTS','VORP','FG','WS','MP','TRB','AST','PER','BPM']#, 'GS'] ### GS
model = RandomForestRegressor(n_estimators = 100, max_features = 'auto', min_samples_leaf = 2)

# Minimum minutes per game
mp = 8
# Propotion of maximum games played
gp = 1/4

In [4]:
# Load training data
train_data = pd.read_csv(path + 'Algorithm/roy/roy_data.csv')

# Load predicted data
data = pd.read_csv(path + f'Algorithm/weekly data/week_{week}.csv') # Load up-to-date data
data = data.groupby('roy').get_group(1)

# Load weekly info
with open(path + f'Algorithm/weekly data/week_{week}.txt', 'r') as file:
            date1 = file.readline().splitlines()[0][7:]
            date2 = file.readline().splitlines()[0][7:]
            n = int(file.readline()[12:])
            
season_progress = n / 15
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,roy,smoy
0,A.J. Green,SG,23,MIL,30,0,9.7,1.6,3.6,0.444,...,4.03,115.27,111.36,3.91,3.97,115.34,111.48,3.86,1,1
10,MarJon Beauchamp,SF,22,MIL,41,9,14.0,2.0,5.0,0.390,...,4.03,115.27,111.36,3.91,3.97,115.34,111.48,3.86,1,1
16,A.J. Lawson,SG,22,DAL,13,0,4.8,1.0,2.0,0.500,...,0.82,118.10,117.29,0.81,0.80,117.97,117.19,0.78,1,1
24,Jaden Hardy,SG,20,DAL,32,0,11.6,2.2,5.1,0.423,...,0.82,118.10,117.29,0.81,0.80,117.97,117.19,0.78,1,1
37,AJ Griffin,SF,19,ATL,59,11,19.8,3.5,7.5,0.472,...,0.05,116.50,116.46,0.04,-0.03,116.69,116.73,-0.04,1,1


In [5]:
# Set training data
X_train = train_data[features]
y_train = train_data['Share']

# Set minimum requirements (games and minutes played)
data = data[data['MP'] >= mp] # Minimum of minutes per game
data = data[data['G'] > season_progress * gp] # Minimum games played

pred = []
for step in range(0, 10) :
    print('Model ' + str(step+1) + '/10 ...')
    clear_output(wait = True)
    # Fit Machine Learning model
    model.fit(X_train, y_train)

    # Predict shares of test data
    pred.append(model.predict(data[features]))

Model 10/10 ...


In [6]:
# Assemble results DataFrame
res = data.assign(Share = [np.array(pred)[:,i].mean() for i in range(0, len(data))])
res = res.sort_values('Share', ascending = False) # Sort values by predictions
res['Rank'] = list(range(1, len(res)+1))
res['week'] = len(res) * [week] # Add week

res = res.reset_index(drop = True)[['Player', 'MP' ,'Tm', 'PER', 'VORP', 'WS', 'FG', 'PTS', 'Seed', 'Share']]
res.head(5)

,Player,MP,Tm,PER,VORP,WS,FG,PTS,Seed,Share
0,Paolo Banchero,33.7,ORL,15.1,0.282759,2.686207,6.6,20.0,13,0.262918
1,Walker Kessler,22.0,UTA,21.1,2.116129,7.406452,3.7,8.4,9,0.176757
2,Bennedict Mathurin,28.1,IND,13.5,-0.869697,2.112121,5.4,16.9,12,0.159103
3,Jalen Duren,25.1,DET,16.5,0.473077,5.519231,3.6,8.5,15,0.097873
4,Mark Williams,17.6,CHO,20.9,0.937143,5.388571,3.5,8.4,14,0.067315


In [7]:
res.to_csv(path + f'Results/roy/results_week_{week}.csv', index = None)